<a href="https://colab.research.google.com/github/JHyunjun/TF2.0_Generative-Adversarial-Network/blob/main/TF2_0_Conditional_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#import

import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import(Activation, BatchNormalization, Concatenate, Dense, Embedding, Flatten, Input, Multiply, Reshape)
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Conv2D, Conv2DTranspose
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam


In [ ]:
img_rows = 28
img_cols = 28
channels = 1

img_shape = (img_rows, img_cols, channels)
z_dim = 100
num_classes = 10 #MNIST 0~9

In [ ]:
#Designing Generator

def build_generator(z_dim) : 
  model = Sequential()
  model.add(Dense(256*7*7, input_dim = z_dim))
  model.add(Reshape((7,7,256)))
  
  model.add(Conv2DTranspose(128, kernel_size = 3, strides = 2, padding = 'same'))
  model.add(BatchNormalization())
  model.add(LeakyReLU(alpha = 0.02))
  
  model.add(Conv2DTranspose(64, kernel_size = 3, strides = 1, padding = 'same'))
  model.add(BatchNormalization())
  model.add(LeakyReLU(alpha = 0.02))

  model.add(Conv2DTranspose(1, kernel_size = 3, strides = 2, padding = 'same'))
  model.add(Activation('tanh'))

  return model

# Designing C-GAN Generator 
# C-GAN의 특성상 Label과 Data를 같이 Input으로 넣어주어야 하는데, CNN기반으로 네트워크를 짤 시, 이미지 사이즈를 훼손하면 안됨. 즉, 이미지에 one hot encoding한 10개 bit값을 넣어준다고 해결되는일이 아님.
# 따라서 이미지 그 자체에 라벨링 값을 녹여버린다는 느낌으로, element multiply를 함

# ① GAN Generator에서 label을 encoding하여 이미지와 동일한 size로 만들고(embedding), Flatten함
# ①과 Image가 동일한 size이므로 Jointed_represntation을함 (element 곱셈)
# Jointed 한 자료에 대해 Generator가 생성한것을 conditioned Image라고하며 이것을 Return함 → 라벨링과 이미지가 혼합된 값을 Input으로 이미지를 Generating한다고 보면됨


def build_cgan_generator(z_dim) : 
  z = Input(shape = (z_dim,)) # (None, z_dim)
  label = Input(shape = (1,), dtype = 'int32') #(None,1)
  
  label_embedding = Embedding(num_classes, z_dim, input_length = 1)(label) #(None, 1, 100) = ([label], z_dim)
  label_embedding = Flatten()(label_embedding) #(None, z_dim) ※ Joint multiply를 하기위해 레이블 데이터를 latent space Z와 동일한 사이즈로 만듦

  jointed_represent = Multiply()([z, label_embedding]) #(None, z_dim) : latent space에 label을 녹여서 합침
  
  generator = build_generator(z_dim) #build_ganerator 함수로 100차원 latent space를 통해 28x28x1 이미지를 만들어내는 모델 구성 = generator

  conditioned_img = generator(jointed_represent) #jointed 값을 input으로 이미지 생성
  
  return Model([z, label], conditioned_img)
  

In [ ]:
#Designing Discriminator

def build_discriminator(img_shape) : 

  model = Sequential()
  
  model.add(Conv2D(64, kernel_size = 3, strides = 2, input_shape = (img_shape[0], img_shape[1], img_shape[2] + 1 ), padding = 'same')) # img_shape : (28,28,1) → 2차원으로 들어옴 (원본이미지,jointed 이미지)
  model.add(LeakyReLU(alpha = 0.02))
  print(img_shape[0])

  model.add(Conv2D(64, kernel_size = 3, strides = 2, padding = 'same'))
  model.add(LeakyReLU(alpha = 0.02))

  model.add(Conv2D(128, kernel_size = 3, strides = 2, padding = 'same'))
  model.add(LeakyReLU(alpha = 0.02))

  model.add(Flatten())
  model.add(Dense(1, activation = 'sigmoid'))

  return model

def build_cgan_discriminator(img_shape) : 

  img = Input(shape = img_shape) #입력된 이미지 (None, 28,28,1)
  label = Input(shape = (1,), dtype = 'int32') #(None, 1) 입력된 이미지의 label

  label_embedding = Embedding(num_classes, np.prod(img_shape), input_length = 1)(label) #(None, 1, 784) ※numpy.prod는 axis에 대한 모든값을 곱한값을 return, 따라서 28x28x1=784를 return
  label_embedding = Flatten()(label_embedding) #(None, 784)
  label_embedding = Reshape(img_shape)(label_embedding) #(None, 28, 28, 1) 
  
  concatenated = Concatenate(axis = -1)([img, label_embedding]) #(None, 28, 28, 2) : [중요한부분] concat함수를 통해 원본이미지와 label+img한 이미지를 각각의 차원으로 둠
  
  discriminator = build_discriminator(img_shape) #Discriminator 네트워크 세팅
  classification = discriminator(concatenated) # 28x28x2 최종 합성 이미지를 확률값으로 return 할것

  return Model([img, label], classification)


In [ ]:
# Integrated G and D

def build_cgan(generator, discriminator) : #generator는 build_cgan_generator(z_dim)임.
  z = Input(shape = (z_dim, ))
  label = Input(shape = (1, ))

  img = generator([z, label]) #레이블에 따른 이미지 Gen
  classification = discriminator([img, label])
  
  model = Model([z, label], classification) #latent space,label을 통해 Generating한 이미지에 대한 Discriminator의 진짜/가짜 판단확률을 return
  return model


In [ ]:
#Training Frame
discriminator = build_cgan_discriminator(img_shape) #cgan Disc 네트워크 세팅
discriminator.compile(loss = 'binary_crossentropy', optimizer = Adam(learning_rate = 0.00001), metrics = ['accuracy']) #Disc 컴파일

generator = build_cgan_generator(z_dim) #Gen 네트워크 세팅
discriminator.trainable = False #Gen 학습시 Disc는 학습되지 않도록 끔

cgan = build_cgan(generator, discriminator) # C-Gen학습
cgan.compile(loss ='binary_crossentropy' ,optimizer = Adam())

In [ ]:
accuracies = []
losses = []


def train(iterations, batch_size, sample_interval):

    # MNIST 데이터셋을 로드합니다.
    (X_train, y_train), (_, _) = mnist.load_data()

    # [0, 255] 사이 흑백 픽셀 값을 [–1, 1]로 스케일 변환합니다.
    X_train = X_train / 127.5 - 1.
    X_train = np.expand_dims(X_train, axis=3)

    # 진짜 이미지의 레이블: 모두 1
    real = np.ones((batch_size, 1))

    # 가짜 이미지의 레이블: 모두 0
    fake = np.zeros((batch_size, 1))

    for iteration in range(iterations):

        # -------------------------
        #  판별자를 훈련합니다.
        # -------------------------

        # 진짜 이미지와 레이블로 이루어진 랜덤한 배치를 얻습니다.
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        imgs, labels = X_train[idx], y_train[idx]

        # 가짜 이미지 배치를 생성합니다.
        z = np.random.normal(0, 1, (batch_size, z_dim))
        gen_imgs = generator.predict([z, labels])

        # 판별자를 훈련합니다.
        d_loss_real = discriminator.train_on_batch([imgs, labels], real)
        d_loss_fake = discriminator.train_on_batch([gen_imgs, labels], fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # ---------------------
        #  생성자를 훈련합니다.
        # ---------------------

        # 잡음 벡터의 배치를 생성합니다.
        z = np.random.normal(0, 1, (batch_size, z_dim))

        # 랜덤한 레이블의 배치를 얻습니다.
        labels = np.random.randint(0, num_classes, batch_size).reshape(-1, 1)

        # 생성자를 훈련합니다.
        g_loss = cgan.train_on_batch([z, labels], real)

        if (iteration + 1) % sample_interval == 0:

            # 훈련 과정을 출력합니다.
            print("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" %
                  (iteration + 1, d_loss[0], 100 * d_loss[1], g_loss))

            # 훈련이 끝난 후 그래프를 그리기 위해 손실과 정확도를 저장합니다.
            losses.append((d_loss[0], g_loss))
            accuracies.append(100 * d_loss[1])

            # 생성한 이미지 샘플을 출력합니다.
            sample_images()

In [ ]:
def sample_images(image_grid_rows=2, image_grid_columns=5):

    # 랜덤한 잡음을 샘플링합니다.
    z = np.random.normal(0, 1, (image_grid_rows * image_grid_columns, z_dim))

    # 0–9 사이의 이미지 레이블을 만듭니다.
    labels = np.arange(0, 10).reshape(-1, 1)

    # 랜덤한 잡음에서 이미지를 생성합니다.
    gen_imgs = generator.predict([z, labels])

    # 이미지 픽셀 값을 [0, 1] 사이로 스케일을 변환합니다.
    gen_imgs = 0.5 * gen_imgs + 0.5

    # 이미지 그리드를 설정합니다.
    fig, axs = plt.subplots(image_grid_rows,
                            image_grid_columns,
                            figsize=(10, 4),
                            sharey=True,
                            sharex=True)

    cnt = 0
    for i in range(image_grid_rows):
        for j in range(image_grid_columns):
            # 이미지 그리드를 출력합니다.
            axs[i, j].imshow(gen_imgs[cnt, :, :, 0], cmap='gray')
            axs[i, j].axis('off')
            axs[i, j].set_title("Digit: %d" % labels[cnt])
            cnt += 1

In [ ]:
iterations = 20000
batch_size = 32
sample_interval = 1000

# 지정된 반복 횟수 동안 CGAN을 훈련합니다.
train(iterations, batch_size, sample_interval)

In [ ]:
image_grid_rows = 10
image_grid_columns = 5

# 랜덤한 잡음을 샘플링합니다.
z = np.random.normal(0, 1, (image_grid_rows * image_grid_columns, z_dim))

# 생성할 이미지 레이블을 5개씩 준비합니다.
labels_to_generate = np.array([[i for j in range(5)] for i in range(10)])
labels_to_generate = labels_to_generate.flatten().reshape(-1, 1)

# 랜덤한 잡음에서 이미지를 생성합니다.
gen_imgs = generator.predict([z, labels_to_generate])

# 이미지 픽셀 값을 [0, 1] 사이로 스케일을 변환합니다.
gen_imgs = 0.5 * gen_imgs + 0.5

# 이미지 그리드를 설정합니다.
fig, axs = plt.subplots(image_grid_rows,
                        image_grid_columns,
                        figsize=(10, 20),
                        sharey=True,
                        sharex=True)

cnt = 0
for i in range(image_grid_rows):
    for j in range(image_grid_columns):
        # 이미지 그리드를 출력합니다.
        axs[i, j].imshow(gen_imgs[cnt, :, :, 0], cmap='gray')
        axs[i, j].axis('off')
        axs[i, j].set_title("Digit: %d" % labels_to_generate[cnt])  ## NEW
        cnt += 1